### Using the results from the AAI analysis create graphs of AAI versus O2 consumption for SAG pairs on individual days.

### Inputs
/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/All_GoM_SAGs_1cell_20kb_decon_531normalized_predresp_rate_GTDBclass.csv
/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/aai/GoM_RSG_aai_outputs/aai/aai_summary.tsv

### Outputs 
/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_images/AAI_vs_log10_O2_ratio*.png graph files

rerun on 2021-12-07 after correcting for salinity of media samples The last bit of this code is not finished

In [1]:
import pandas as pd
from pandas import DataFrame
import os
import sys
import csv
from pathlib import Path
import seaborn as sns
sns.set()
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
plt.show()
from collections import defaultdict
import os.path as op
import glob

os.chdir('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/aai/')
outdir='/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/aai/'
images='/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_images/AAI/'
matplotlib.__version__

'3.3.3'

In [2]:
!head /mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/All_GoM_SAGs_1cell_20kb_decon_531normalized_predresp_rate_GTDBclass.csv

,X2,name,Plate,well_type,wga_cp,wgs_run_id,raw_read_count,final_assembly_length,max_contig_length,gc_content,checkM_estimated_completeness,total_CDS_count,hypothetical_cds_fraction,average_cds_length,number_multi_copy,checkM_classification,SSU_classification_1,SSU_classification_2,SSU_classification_3,notes,eventnum,well2,X531.40..488.,X572.27..488.,X692.40..488.,Trigger.Pulse.Width,SSC,FSC,estimated_diameter,PCA_analysis_Action,PCA_analysis_Note,Location,Sample_collection_Date,FACS_date,gate,depth..m.,sample,slope,intercept,normalized_531_value,Predicted_respiration_rate_fmol_O2perhr,lower_resp_bound,upper_resp_bound,SAG,GTDB_classification,GTDBtk_classification
0,A01,AH-135-A01,AH-135,1 cell,2.131257507,170717_NB502014_0032_AHMHYCAFXX,428328,378833,26163,29.81076094,22.64,390.0,0.241025641,865.8923077,0,k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rickettsiales;f__Pelagibacteraceae;g__Candidatus_Pelagibacter;s__Candidatus_Pelagibacter_ubique;unresolved,no SSU regions found

In [3]:
columns=['name', 'GTDB_classification', 'estimated_diameter', 'Predicted_respiration_rate_fmol_O2perhr',
        'Sample_collection_Date', 'gate', 'FACS_date', 'checkM_estimated_completeness']
SAG_key=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/All_GoM_SAGs_1cell_20kb_decon_531normalized_predresp_rate_GTDBclass.csv',
                   usecols=columns)
SAG_key=SAG_key[SAG_key['gate']=='All prokaryotes (SYTO-9-positive)']
SAG_key=SAG_key[SAG_key['Sample_collection_Date']==SAG_key['FACS_date']]
SAG_key=SAG_key[SAG_key['checkM_estimated_completeness']>=50]
SAG_list=SAG_key['name'].tolist()
SAG_key

,name,checkM_estimated_completeness,estimated_diameter,Sample_collection_Date,FACS_date,gate,Predicted_respiration_rate_fmol_O2perhr,GTDB_classification
287,AH-141-A07,52.83,0.29,170412,170412,All prokaryotes (SYTO-9-positive),NaN,Pelagibacter
296,AH-141-A21,79.42,0.30,170412,170412,All prokaryotes (SYTO-9-positive),NaN,Pelagibacter
297,AH-141-A22,65.66,0.35,170412,170412,All prokaryotes (SYTO-9-positive),NaN,Pelagibacter
302,AH-141-B06,53.63,0.43,170412,170412,All prokaryotes (SYTO-9-positive),NaN,Hel1-33-131
310,AH-141-B20,62.03,0.37,170412,170412,All prokaryotes (SYTO-9-positive),NaN,Porticoccus
...,...,...,...,...,...,...,...,...
5828,AH-273-O22,58.79,0.51,171102,171102,All prokaryotes (SYTO-9-positive),NaN,GCA-2691785
5843,AH-273-P16,67.60,0.33,171102,171102,All prokaryotes (SYTO-9-positive),NaN,UBA10364
5845,AH-273-P18,67.28,0.41,171102,171102,All prokaryotes (SYTO-9-positive),NaN,MAG-121220-bin8
5846,AH-273-P19,55.29,0.34,171102,171102,All prokaryotes (SYTO-9-positive),NaN,UBA3537


In [6]:
#read result file from RSG_ANI analysis and parse it to remove AAI=0 and add SAG identifiers

aai=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/aai/GoM_Syto9_outputs/aai/aai_summary.tsv',
                sep='\t')
aai=aai[aai['Mean AAI'] != 0.0]
aai['SAG_A']=aai['Genome A'].str[:10]
aai['SAG_B']=aai['Genome B'].str[:10]

aai=aai[aai['SAG_A'].isin(SAG_list)]
aai=aai[aai['SAG_B'].isin(SAG_list)]


In [8]:
aai

,Genome A,Genes in A,Genome B,Genes in B,# orthologous genes,Mean AAI,Std AAI,Orthologous fraction (OF),SAG_A,SAG_B
8605,AH-660-L02_contigs,1126,AH-229-N19_contigs,1154,139,41.17,8.20,12.34,AH-660-L02,AH-229-N19
8622,AH-660-L02_contigs,1126,AH-229-K22_contigs,1205,131,42.55,7.74,11.63,AH-660-L02,AH-229-K22
8630,AH-660-L02_contigs,1126,AH-273-P20_contigs,1052,123,43.59,9.71,11.69,AH-660-L02,AH-273-P20
8632,AH-660-L02_contigs,1126,AH-541-O07_contigs,937,104,44.71,9.68,11.10,AH-660-L02,AH-541-O07
8633,AH-660-L02_contigs,1126,AH-660-L16_contigs,942,99,43.94,9.43,10.51,AH-660-L02,AH-660-L16
...,...,...,...,...,...,...,...,...,...,...
1519509,AH-660-J19_contigs,1145,AH-141-C19_contigs,1477,132,42.75,8.12,11.53,AH-660-J19,AH-141-C19
1519512,AH-660-J19_contigs,1145,AH-229-C19_contigs,885,578,78.13,12.84,65.31,AH-660-J19,AH-229-C19
1519516,AH-660-J19_contigs,1145,AH-541-G04_contigs,1574,300,47.49,11.39,26.20,AH-660-J19,AH-541-G04
1519521,AH-660-J19_contigs,1145,AH-273-E10_contigs,1399,369,46.69,10.93,32.23,AH-660-J19,AH-273-E10


In [7]:
# This bit of code was not finished.

Syto9hist=sns.histplot(data=aai, x='Mean AAI', ax=ax, binwidth=1)
AAIhist=AAIhist.set_title("Sampling Date "+i)
fig=AAIhist.get_figure()
fig.savefig(images+'SYTO9_AAI_histogram_'+".pdf")

NameError: name 'df' is not defined